<b>Create a python Flask WebApp

<b>Build an image from this App

In [1]:
!docker images

REPOSITORY   TAG       IMAGE ID       CREATED        SIZE
myimage      v1        b05e18118ab2   23 hours ago   769MB
ubuntu       v2        b49d7a0ddb93   3 days ago     77.8MB
busybox      latest    7cfbbec8963d   4 weeks ago    4.86MB
ubuntu       latest    08d22c0ceb15   5 weeks ago    77.8MB


In [ ]:
!docker build -t python-flask-app:v1 .

In [3]:
!docker images

REPOSITORY         TAG       IMAGE ID       CREATED              SIZE
python-flask-app   v1        3efa6b7401b6   About a minute ago   942MB
myimage            v1        b05e18118ab2   23 hours ago         769MB
ubuntu             v2        b49d7a0ddb93   3 days ago           77.8MB
busybox            latest    7cfbbec8963d   4 weeks ago          4.86MB
ubuntu             latest    08d22c0ceb15   5 weeks ago          77.8MB


<b>Create container from this image and run

In [4]:
!docker ps -a

CONTAINER ID   IMAGE        COMMAND                  CREATED        STATUS                    PORTS     NAMES
0bef93429cef   myimage:v1   "httpd /bin/sh -c '[…"   23 hours ago   Exited (1) 23 hours ago             interesting_perlman
b90dfb638d76   b49          "/bin/bash"              25 hours ago   Exited (0) 25 hours ago             objective_mayer


In [ ]:
!docker run -dp 5002:5000 \
    --name python-flask-app-container \
    -v "$(pwd):/app" 3efa6b7401b6

In [5]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED         STATUS         PORTS                                       NAMES
434252af2c52   3efa6b7401b6   "python -m flask run…"   6 seconds ago   Up 3 seconds   0.0.0.0:5002->5000/tcp, :::5002->5000/tcp   python-flask-app-container


In [6]:
!docker logs 434

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.17.0.2:5000
Press CTRL+C to quit


In [7]:
!curl localhost:5002

Python Flask With Docker Practice

<b> Update code -> docker stop container -> docker start container

In [ ]:
!docker restart container

<b> Update code, add MySQL container, add Network, Volume

In [8]:
!docker volume create mysql-python-app-volume

mysql-python-app-volume


In [9]:
!docker volume create mysql-python-app-volume-config

mysql-python-app-volume-config


In [10]:
!docker volume ls

DRIVER    VOLUME NAME
local     D2
local     DISK1
local     data
local     mysql-python-app-volume
local     mysql-python-app-volume-config


In [11]:
!docker network create mysql-python-app-network

ea4c4a5504927436c2d3a8f5a5c6eb957f7d8db4cbbb6ea7218052c6abfcbd59


In [12]:
!docker network ls

NETWORK ID     NAME                       DRIVER    SCOPE
097fe9392588   bridge                     bridge    local
dbc5ffc40741   host                       host      local
3b8cd3952397   mynetwork                  bridge    local
ea4c4a550492   mysql-python-app-network   bridge    local
7c7c98726ec1   none                       null      local


<b>Create mysql container

In [14]:
# --rm: automatic remove container after stop
!docker run --rm -d\
    -v mysql-python-app-volume:/var/lib/mysql \
    -v mysql-python-app-volume-config:/etc/mysql \
    --name mysql-flask-app-container \
    -p 3309:3306 \
    --network mysql-python-app-network \
    -e MYSQL_ROOT_PASSWORD=Abc@123456789 \
    mysql:8.0.28

Unable to find image 'mysql:8.0.28' locally
8.0.28: Pulling from library/mysql

15f6562f: Pulling fs layer 
eb237a1b: Pulling fs layer 
ac85066b: Pulling fs layer 
524f6c89: Pulling fs layer 
8631064f: Pulling fs layer 
3ec3ff50: Pulling fs layer 
dc337dcb: Pulling fs layer 
a78d12d6: Pulling fs layer 
a07a253d: Pulling fs layer 
05dc9e58: Pulling fs layer 
41026e44: Pulling fs layer 
f73ebe32: Pull complete  121B/121B5kBBDigest: sha256:fc77d54cacef90ad3d75964837fad0f2a9a368b69e7d799665a3f4e90e600c2d
Status: Downloaded newer image for mysql:8.0.28
8dfab3f0fd15c81666aca0079518fcdd4586e686a94923c9bd1c000154967657


In [16]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED          STATUS          PORTS                                                  NAMES
8dfab3f0fd15   mysql:8.0.28   "docker-entrypoint.s…"   29 seconds ago   Up 22 seconds   33060/tcp, 0.0.0.0:3309->3306/tcp, :::3309->3306/tcp   mysql-flask-app-container
434252af2c52   3efa6b7401b6   "python -m flask run…"   21 minutes ago   Up 21 minutes   0.0.0.0:5002->5000/tcp, :::5002->5000/tcp              python-flask-app-container


<b>Access mysql's command line inside mysql-flask-app-container

<b>docker exec</b> executes a new command / create a new process in the container’s environment, while <b>docker attach</b> just connects the standard input/output/error of the main process(with PID 1) inside the container to corresponding standard input/output/error of current terminal(the terminal you are using to run the command).

In [ ]:
!docker exec -it mysql-flask-app-container mysql -u root -p

In [ ]:
!show databases;

<b>Create Database

In [1]:
!docker restart 434

434


In [2]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED          STATUS          PORTS                                                  NAMES
8dfab3f0fd15   mysql:8.0.28   "docker-entrypoint.s…"   21 minutes ago   Up 20 minutes   33060/tcp, 0.0.0.0:3309->3306/tcp, :::3309->3306/tcp   mysql-flask-app-container
434252af2c52   3efa6b7401b6   "python -m flask run…"   41 minutes ago   Up 2 seconds    0.0.0.0:5002->5000/tcp, :::5002->5000/tcp              python-flask-app-container


<b> Add Networking

In [3]:
!docker rm -f 434

434


In [2]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED          STATUS          PORTS                                                  NAMES
8dfab3f0fd15   mysql:8.0.28   "docker-entrypoint.s…"   25 minutes ago   Up 25 minutes   33060/tcp, 0.0.0.0:3309->3306/tcp, :::3309->3306/tcp   mysql-flask-app-container


In [3]:
!docker run -dp 5002:5000 \
    --name python-flask-app-container \
    --network mysql-python-app-network \
    -v "$(pwd):/app"\
    3efa6b7401b6

b9d0cea0474ef894eafcb4a147b3545de47499d6ab2b9e0e9b6bbe617a484702


In [4]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED          STATUS          PORTS                                                  NAMES
b9d0cea0474e   3efa6b7401b6   "python -m flask run…"   7 seconds ago    Up 4 seconds    0.0.0.0:5002->5000/tcp, :::5002->5000/tcp              python-flask-app-container
8dfab3f0fd15   mysql:8.0.28   "docker-entrypoint.s…"   26 minutes ago   Up 25 minutes   33060/tcp, 0.0.0.0:3309->3306/tcp, :::3309->3306/tcp   mysql-flask-app-container


In [6]:
!docker logs b9d

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.0.3:5000
Press CTRL+C to quit


In [7]:
!curl localhost:5002/initdb

init Database

In [8]:
!docker logs b9d

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.0.3:5000
Press CTRL+C to quit
172.18.0.1 - - [18/Apr/2023 04:59:18] "GET /initdb HTTP/1.1" 200 -


In [ ]:
!docker exec -it mysql-flask-app-container mysql -u root -p

In [ ]:
!show database;

<b> Init table

In [9]:
!docker restart b9d

b9d


In [10]:
!curl localhost:5002/init_tables

init_tables

In [11]:
!docker logs b9d

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.0.3:5000
Press CTRL+C to quit
172.18.0.1 - - [18/Apr/2023 04:59:18] "GET /initdb HTTP/1.1" 200 -
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.0.3:5000
Press CTRL+C to quit
172.18.0.1 - - [18/Apr/2023 05:05:13] "GET /init_tables HTTP/1.1" 200 -


In [ ]:
!use ProductManagement;

In [ ]:
!show tables;

In [ ]:
!describe tblProduct;

In [ ]:
!INSERT INTO tblProduct(name, description) VALUES
('macboook pro m1', 'please buy'),
('iphone 13', 'please buy and buy');

In [ ]:
!select * from tblProduct;

In [13]:
!docker restart b9d

b9d


In [15]:
!curl localhost:5002/products

[{"id": 1, "name": "macboook pro m1", "description": "please buy"}, {"id": 2, "name": "iphone 13", "description": "please buy and buy"}]

<b> Docker Compose </b><br>

In [19]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED       STATUS       PORTS                                                  NAMES
b9d0cea0474e   3efa6b7401b6   "python -m flask run…"   2 hours ago   Up 2 hours   0.0.0.0:5002->5000/tcp, :::5002->5000/tcp              python-flask-app-container
8dfab3f0fd15   mysql:8.0.28   "docker-entrypoint.s…"   2 hours ago   Up 2 hours   33060/tcp, 0.0.0.0:3309->3306/tcp, :::3309->3306/tcp   mysql-flask-app-container


In [20]:
!docker rm -f b9d

b9d


In [21]:
!docker rm -f 8df

8df


In [ ]:
!docker-compose -f ./docker-compose.dev.yml up --build -d

In [22]:
!docker ps

CONTAINER ID   IMAGE                 COMMAND                  CREATED          STATUS          PORTS                                                  NAMES
b6e21c291154   python-flask-app:v1   "python -m flask run…"   31 seconds ago   Up 28 seconds   0.0.0.0:5002->5000/tcp, :::5002->5000/tcp              python-flask-app-container
94c42eb4cf85   mysql:8.0.28          "docker-entrypoint.s…"   31 seconds ago   Up 28 seconds   33060/tcp, 0.0.0.0:3309->3306/tcp, :::3309->3306/tcp   mysql-flask-app-container


In [25]:
!curl localhost:5002/products

<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>


In [26]:
!docker logs b6e

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.0.3:5000
Press CTRL+C to quit
[2023-04-18 07:08:44,926] ERROR in app: Exception on /products [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/mysql/connector/network.py", line 574, in open_connection
    self.sock.connect(sockaddr)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/pytho

In [27]:
!curl localhost:5002/initdb

init Database

In [28]:
!curl localhost:5002/init_tables

init Tables

In [ ]:
!docker exec -it mysql-flask-app-container mysql -u root -p

In [ ]:
!INSERT INTO tblProduct(name, description) VALUES
('macboook pro m1', 'please buy'),
('iphone 13', 'please buy and buy');

In [29]:
!curl localhost:5002/products

[{"id": 1, "name": "macboook pro m1", "description": "please buy"}, {"id": 2, "name": "iphone 13", "description": "please buy and buy"}]